In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import pandas as pd


In [55]:
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_no_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_apply_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\cleaned_combine_all_class_with_label_applied_fft.csv'
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win5_lab50.csv'
df = pd.read_csv(file_path)

In [56]:
# -- Prepare for modeling --
# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
#    - We'll drop 'start_time' because it's just a reference
#    - We'll keep 'label_encoded' as the target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,   # 20% for testing
    random_state=42  # for reproducibility
)

# Apply min-max normalization (0-1) to these columns
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# 4) Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5) Evaluate
y_pred = model.predict(scaler.transform(X_test))
print(classification_report(y_test, y_pred, target_names=le.classes_))

                             precision    recall  f1-score   support

            climbing_stairs       0.84      0.72      0.78        53
          descending_stairs       0.90      0.88      0.89        73
                    nothing       1.00      0.93      0.96        54
                    running       0.98      1.00      0.99        53
sitting_standing_transition       0.93      0.98      0.95        64
                    walking       0.86      0.97      0.91        67

                   accuracy                           0.91       364
                  macro avg       0.92      0.91      0.91       364
               weighted avg       0.92      0.91      0.91       364



In [ ]:
!pip install tensorflow onnx onnxmltools skl2onnx nyoka scikit-learn joblib pandas numpy onnxruntime jpmml-evaluator

^C


In [175]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
import onnx
import onnxmltools
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from nyoka import skl_to_pmml
import json

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# -- Load Data --
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win2_lab50.csv'
df = pd.read_csv(file_path)

# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_test_raw = X_test
X_train_raw = X_train
# 4) Normalize features (0-1 scaling)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5) Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)


# 7) Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))


import joblib
# Save the trained model
joblib.dump(model, "rf_model.pkl")
# Save the scaler
joblib.dump(scaler, "scaler.pkl")
# Save the label encoder
joblib.dump(le, "label_encoder.pkl")

print("Model, Scaler, and Label Encoder have been saved successfully.")

                             precision    recall  f1-score   support

            climbing_stairs       0.85      0.78      0.82       170
          descending_stairs       0.83      0.81      0.82       158
                    nothing       1.00      0.91      0.95       148
                    running       1.00      1.00      1.00       145
sitting_standing_transition       0.91      0.99      0.94       146
                    walking       0.85      0.94      0.89       154

                   accuracy                           0.90       921
                  macro avg       0.91      0.91      0.90       921
               weighted avg       0.90      0.90      0.90       921

Model, Scaler, and Label Encoder have been saved successfully.


In [176]:
# Create a dictionary mapping encoded labels to original labels
label_mapping = {index: label for index, label in enumerate(le.classes_)}

# Print the mapping
print("Label Mapping:", label_mapping)

Label Mapping: {0: 'climbing_stairs', 1: 'descending_stairs', 2: 'nothing', 3: 'running', 4: 'sitting_standing_transition', 5: 'walking'}


In [6]:
# IF USE TORCH
# import torch
# import torch.nn as nn
# import torch.optim as optim

# class RF_NN(nn.Module):
#     def __init__(self, input_size, num_classes):
#         super(RF_NN, self).__init__()
#         self.fc1 = nn.Linear(input_size, 160)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(160, num_classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         x = self.softmax(x)
#         return x

# # Create the PyTorch model
# input_size = X_train.shape[1]
# num_classes = len(set(y))
# torch_model = RF_NN(input_size, num_classes)


In [152]:
import random
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -- Load Data --
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win2_lab50.csv'
df = pd.read_csv(file_path)

# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_test_raw = X_test
X_train_raw = X_train
# 4) Normalize features (0-1 scaling)
scaler = MinMaxScaler()
scaler.fit(X_train)

# Convert integer labels to one-hot encoded labels for MLP training
num_classes = len(le.classes_)
y_train_ohe = to_categorical(y_train, num_classes=num_classes)
y_test_ohe = to_categorical(y_test, num_classes=num_classes)

# Create constant tensors for the normalization parameters
scaler_scale = tf.constant(scaler.scale_, dtype=tf.float32)
scaler_min = tf.constant(scaler.min_, dtype=tf.float32)

def rf_to_nn_with_normalization(activation_fn:str):
    return tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train_raw.shape[1],)),
        tf.keras.layers.Lambda(lambda x: x * scaler_scale + scaler_min, name="Normalization"),
        tf.keras.layers.Dense(85, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(65, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(45, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(35, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(25, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(15, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(10, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(num_classes, activation="softmax",
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED))
    ])

# Create and compile the model with normalization incorporated
tf_model = rf_to_nn_with_normalization('relu')
tf_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
tf_model.summary()

# Train the model using one-hot encoded labels
# Note: Since the model applies internal normalization, we feed raw features.
tf_model.fit(
    X_train_raw, 
    y_train_ohe,  
    epochs=63, 
    verbose=1
)

# Save the Keras model in the new .keras format
keras_model_path = "win2_lab50_acc93.keras"
tf_model.save(keras_model_path, save_format="keras")

print(f"Keras model saved to {keras_model_path}")


Model: "sequential_57"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Normalization (Lambda)          │ (None, 51)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_329 (Dense)               │ (None, 85)             │         4,420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_330 (Dense)               │ (None, 65)             │         5,590 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_331 (Dense)               │ (None, 45)             │         2,970 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_332 (Dense)               │ (None, 35)             │         1,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_333 (Dense)               │ (None, 25)             │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_334 (Dense)               │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_335 (Dense)               │ (None, 10)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_336 (Dense)               │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,106 (62.91 KB)

 Trainable params: 16,106 (62.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3752 - loss: 1.5995
Epoch 2/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6526 - loss: 0.7866
Epoch 3/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7619 - loss: 0.5662
Epoch 4/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8000 - loss: 0.4898
Epoch 5/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8175 - loss: 0.4494
Epoch 6/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8332 - loss: 0.4219
Epoch 7/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8391 - loss: 0.4010
Epoch 8/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8454 - loss: 0.3847
Epoch 9/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8572 - loss: 0.3695
Epoch 10/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8669 - loss: 0.3517
Epoch 11/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8701 - loss: 0.3413
Epoch 12/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Keras model saved to win2_lab50_acc93.keras


In [174]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)

# -- Load Data --
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win2_lab50.csv'
df = pd.read_csv(file_path)

# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1).values
y = df['label_encoded'].values

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

# 4) Normalize features (0-1 scaling)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define MLPClassifier with custom hidden layers
mlp_sklearn = MLPClassifier(
    hidden_layer_sizes=(80, 60, 40, 30, 20, 15, 10),  # Custom architecture
    activation='relu',  # Activation function
    solver='adam',  # Optimizer (same as PyTorch version)
    max_iter=500,  # Number of epochs
    random_state=SEED,
    verbose=True,
    early_stopping=False,
    alpha=0.001
)

# Train the model
mlp_sklearn.fit(X_train, y_train)

# Predict on the test set
y_pred = mlp_sklearn.predict(X_test)

# Evaluate the model
print(f"Training Accuracy: {mlp_sklearn.score(X_train, y_train):.4f}")
print(f"Test Accuracy: {mlp_sklearn.score(X_test, y_test):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


Iteration 1, loss = 1.85932518
Iteration 2, loss = 1.75775949
Iteration 3, loss = 1.59641079
Iteration 4, loss = 1.48893446
Iteration 5, loss = 1.38226889
Iteration 6, loss = 1.28167120
Iteration 7, loss = 1.18083430
Iteration 8, loss = 1.05474521
Iteration 9, loss = 0.86794332
Iteration 10, loss = 0.66892382
Iteration 11, loss = 0.57176436
Iteration 12, loss = 0.54954714
Iteration 13, loss = 0.53240748
Iteration 14, loss = 0.51496153
Iteration 15, loss = 0.49607306
Iteration 16, loss = 0.48624464
Iteration 17, loss = 0.48081737
Iteration 18, loss = 0.46260065
Iteration 19, loss = 0.44839538
Iteration 20, loss = 0.42862581
Iteration 21, loss = 0.41745226
Iteration 22, loss = 0.40013400
Iteration 23, loss = 0.38431289
Iteration 24, loss = 0.37570049
Iteration 25, loss = 0.39143806
Iteration 26, loss = 0.41133117
Iteration 27, loss = 0.35307296
Iteration 28, loss = 0.35286919
Iteration 29, loss = 0.34207319
Iteration 30, loss = 0.33600760
Iteration 31, loss = 0.32739526
Iteration 32, los

In [160]:
# ✅ **Test TFLite Model**
import tensorflow.lite as tflite

tflite_model_path = "random_forest.tflite"
interpreter = tflite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

tflite_preds = []
for sample in X_test_raw.values.astype(np.float32):
    sample = np.expand_dims(sample, axis=0)
    interpreter.set_tensor(input_index, sample)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    tflite_preds.append(np.argmax(pred))

print("TFLite Model Performance:\n", classification_report(y_test, tflite_preds, target_names=le.classes_))



TFLite Model Performance:
                              precision    recall  f1-score   support

            climbing_stairs       0.85      0.89      0.87       170
          descending_stairs       0.87      0.87      0.87       158
                    nothing       1.00      0.93      0.96       148
                    running       1.00      0.99      0.99       145
sitting_standing_transition       0.91      0.97      0.94       146
                    walking       0.94      0.92      0.93       154

                   accuracy                           0.93       921
                  macro avg       0.93      0.93      0.93       921
               weighted avg       0.93      0.93      0.93       921



In [ ]:
import numpy as np

def compute_lpf_coefficients(N, wc):
    midpoint = N / 2
    h = []
    for n in range(N + 1):
        if n == midpoint:
            #add wc/pi
            h.append(wc/np.pi)
        else:
            #add FIR coefficient
            result = (wc/np.pi) * (np.sin(wc*(n-(N/2)))/(wc*(n-(N/2))))
            h.append(result)
    return np.array(h)
    
# Parameters
h_fqu_cut = 15   # Cutoff frequency (Hz)
l_feq_cut = 0.4   # Cutoff frequency (Hz)
fs = 50 # sampling rate on Hz
N = 100      # Filter order
wc_h = 2*np.pi*(h_fqu_cut/fs) # Normalized cutoff angular frequency
wc_l = 2*np.pi*(l_feq_cut/fs) # Normalized cutoff angular frequency

# Compute the BPF coefficients
h_lpf_150 = compute_lpf_coefficients(N, wc_h)
h_lpf_100 = compute_lpf_coefficients(N, wc_l)
h_bpf = h_lpf_150-h_lpf_100
print(h_bpf.tolist())

example_signal = fft_df.iloc[1, 2:].values
filtered_signal = np.convolve(example_signal, h_bpf, mode='same')  # Convolve with HPF coefficients
filtered_signal

[-0.0037419571351546255, -0.010255735861780687, -0.0005187563533817045, -0.0007779271309784636, -0.011684530606556422, -0.005450266280245839, 0.0010814544484847614, -0.010499645183615583, -0.010953739740245078, 0.0005338700387511322, -0.007200384902161859, -0.015312613124998927, -0.002975404036703231, -0.0031894352651102202, -0.01700029660480256, -0.00893347837083537, -0.0003689109478433577, -0.015278091894180038, -0.015786924220581665, -0.0005017439457106806, -0.010589392477812825, -0.021345068779959075, -0.004534786922936576, -0.004591712904678152, -0.023462354270661682, -0.012109227658250444, 0.0002218563385282049, -0.020801224793278724, -0.021437109942985158, 0.0012258380737344265, -0.013437896293417148, -0.029609449759968987, -0.003510306921295012, -0.003116930258726445, -0.033250760949093114, -0.01452654075837359, 0.0069118844267535475, -0.02927754466342892, -0.030638764282488667, 0.012323855887878953, -0.015334695717194022, -0.04909655126534018, 0.007814968676430655, 0.011056372

array([np.float64(-2.4572991291393254), np.float64(-2.938421045617125),
       np.float64(-3.5433295666435347), np.float64(-2.9608317019068386),
       np.float64(-3.005196457714609), np.float64(-3.902374115708245),
       np.float64(-3.5705299697139714), np.float64(-3.0938659492508944),
       np.float64(-4.0438263501567215), np.float64(-4.234147669793814),
       np.float64(-3.3393399535995556), np.float64(-3.958773310232863),
       np.float64(-4.821588823467398), np.float64(-3.830029707736495),
       np.float64(-3.733336910151764), np.float64(-5.162674625299965),
       np.float64(-4.595645164817652), np.float64(-3.524083143938681),
       np.float64(-5.087626916533736), np.float64(-5.580962821200157),
       np.float64(-3.5832988776013037), np.float64(-4.410217255507737),
       np.float64(-6.6099571012472795), np.float64(-4.461708871545995),
       np.float64(-2.562844875156558), np.float64(-4.055568889156495),
       np.float64(1.1721884624435246), np.float64(13.932130311833223